In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BertConfig, BertForSequenceClassification, BertTokenizer
from datasets import load_dataset
from tqdm import tqdm
import random
from copy import deepcopy
import torch.nn.functional as F

# --- Classes Utilitaires ---

class FeatureHook:
    """Intercepte les activations d'une couche specifique."""
    def __init__(self, module):
        self.hook = module.register_forward_hook(self.hook_fn)
        self.features = None

    def hook_fn(self, module, input, output):
        # Pour les transformers, output peut etre un tuple
        if isinstance(output, tuple):
            self.features = output[0]
        else:
            self.features = output

    def close(self):
        self.hook.remove()

class SimpleGMM:
    """Simple Gaussian Mixture Model using K-means initialization."""
    def __init__(self, n_components, n_features, n_iter=100):
        self.n_components = n_components
        self.n_features = n_features
        self.n_iter = n_iter
        self.mu = None  # [n_components, n_features]

    def fit(self, X):
        """Fit GMM using K-means."""
        device = X.device
        n_samples = X.shape[0]

        # Initialize centroids randomly
        indices = torch.randperm(n_samples)[:self.n_components]
        self.mu = X[indices].clone()

        # K-means iterations
        for _ in range(self.n_iter):
            # Assign samples to nearest centroid
            distances = torch.cdist(X, self.mu)
            assignments = torch.argmin(distances, dim=1)

            # Update centroids
            new_mu = torch.zeros_like(self.mu)
            for k in range(self.n_components):
                mask = assignments == k
                if mask.sum() > 0:
                    new_mu[k] = X[mask].mean(dim=0)
                else:
                    new_mu[k] = self.mu[k]
            self.mu = new_mu

        return self

    def predict(self, X):
        """Predict cluster assignments."""
        distances = torch.cdist(X, self.mu)
        return torch.argmin(distances, dim=1)


# --- DeepSigns Module ---

class DeepSignsModule(nn.Module):
    """Learnable GMM means that project to watermark space."""
    def __init__(self, gmm_mu):
        super().__init__()
        self.var_param = nn.Parameter(gmm_mu.clone(), requires_grad=True)

    def forward(self, matrix_a):
        matrix_g = torch.sigmoid(self.var_param @ matrix_a)
        return matrix_g



class DeepSignSentiment:
    def __init__(self, model_id="prajjwal1/bert-tiny", device="cuda"):
        self.device = device
        self.model_id = model_id
        
        # Chargement du modele
        # self.model = AutoModelForSequenceClassification.from_pretrained(
        #     model_id, num_labels=2, use_safetensors=True
        # ).to(device)
        # self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        config = BertConfig.from_pretrained(model_id, num_labels=2)
        self.model = BertForSequenceClassification.from_pretrained(model_id, config=config).to(device)
        self.tokenizer=BertTokenizer.from_pretrained(model_id)
        # Configuration par defaut
        self.config = {
            "layer_name": "bert.encoder.layer.1.output",  # Derniere couche encoder
             # "layer_name": "classifier",
            "watermark_len": 64,
            "n_components": 5,      # Nombre de clusters GMM
            "nb_wat_classes": 2,     # Nombre de classes porteuses
            "trigger_size": 256,     # Taille du trigger set
            "lr": 5e-5,
            "lr_ds": 1e-3,           # Learning rate for DeepSigns module
            "lambda_1": 0.1,         # GMM loss weight
            "lambda_2": 5.0,         # Watermark loss weight
            "epochs": 30,
        }
        
        self.saved_keys = {}
        
    def list_layers(self):
        """Affiche les couches disponibles pour le watermarking."""
        print("Couches disponibles (modules):")
        for name, module in self.model.named_modules():
            if name:
                print(f"  {name}")
    
    def _get_target_layer(self, model, layer_name):
        """Recupere le module PyTorch correspondant au nom."""
        for name, module in model.named_modules():
            if name == layer_name:
                return module
        raise ValueError(f"Couche {layer_name} introuvable.")
    def _extract_activations(self, model, dataloader, max_samples=1000):
        """
        Extract activations from the target layer for GMM fitting.
        """
        target_layer = self._get_target_layer(model, self.config["layer_name"])
        hook = FeatureHook(target_layer)

        activations = []
        inputs_list =  {
            "input_ids": [],
            "attention_mask": []
        }

        model.eval()
        n_samples = 0

        with torch.no_grad():
            for batch in dataloader:
                if n_samples >= max_samples:
                    break
                input_ids = batch["input_ids"].to(self.device)
                attention_mask = batch["attention_mask"].to(self.device)
                labels = batch["labels"].to(self.device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask)

                # Global average pooling and flatten
                act = hook.features
                print(act.shape)
                if len(act.shape) == 4:
                    act = act.mean(dim=(2, 3))  # [B, C]
                elif len(act.shape) == 3:
                    act = act.mean(dim=(1))
                activations.append(act.cpu())
                inputs_list["input_ids"].append(input_ids.cpu())
                inputs_list["attention_mask"].append(attention_mask.cpu())

                n_samples += input_ids.size(0)



        hook.close()

        activations = torch.cat(activations, dim=0)[:max_samples]
        inputs_list["input_ids"] = torch.cat(inputs_list["input_ids"], dim=0)[:max_samples].to(self.device)
        inputs_list["attention_mask"] = torch.cat(inputs_list["attention_mask"], dim=0)[:max_samples].to(self.device)

        return activations.to(self.device), inputs_list

    def _mu_loss(self, act, mu, mu_bar, watermarked_classes, y_key):
        """
        Compute GMM loss:
        - Minimize distance between carrier class means and statistical means
        - Maximize distance between carrier and non-carrier means
        """
        # Compute statistical means for each watermarked class
        stat_means = torch.stack([
            act[y_key == t].mean(dim=0) if (y_key == t).sum() > 0 else mu[i]
            for i, t in enumerate(watermarked_classes)
        ])

        # Loss to approach GMM means to statistical means
        gmm_loss = F.mse_loss(stat_means, mu, reduction='sum')

        # Loss to separate carrier and non-carrier means
        if len(mu_bar) > 0:
            sep_loss = F.mse_loss(
                mu.unsqueeze(1).expand(-1, len(mu_bar), -1),
                mu_bar.unsqueeze(0).expand(len(mu), -1, -1),
                reduction='mean'
            )
        else:
            sep_loss = torch.tensor(0.0, device=self.device)

        return gmm_loss, sep_loss


    
    def embed(self, train_dataloader, eval_dataloader, trigger_dataset):
        """
        Incorpore la marque DeepSign pendant le finetuning.
        """
        print(f"--- Demarrage Embedding DeepSign ({self.config['layer_name']}) ---")
        
        watermarked_model = self.model.to(self.device)
        watermarked_model.train()
        
       # 1. Extract activations for GMM
        print("Extracting activations for GMM...")
        activations, trigger_inputs= self._extract_activations(
            watermarked_model, train_dataloader, max_samples=self.config["trigger_size"]
        )
        n_features = activations.shape[1]
        print(f"Activation shape: {activations.shape}")

        # 2. Fit GMM
        print("Fitting GMM...")
        gmm = SimpleGMM(self.config["n_components"], n_features)
        gmm.fit(activations)

        # 3. Select watermarked classes (carrier classes)
        y_gmm = gmm.predict(activations)
        unique_classes = torch.unique(y_gmm).tolist()
        n_wat = min(self.config["nb_wat_classes"], len(unique_classes))
        watermarked_classes = torch.tensor(random.sample(unique_classes, n_wat), device=self.device)
        print(f"Watermarked classes: {watermarked_classes.tolist()}")

        # 4. Create trigger set (samples from watermarked classes)
        trigger_mask = torch.isin(y_gmm, watermarked_classes)
        x_key ={"input_ids": trigger_inputs["input_ids"][trigger_mask].to(self.device),
    "attention_mask": trigger_inputs["attention_mask"][trigger_mask].to(self.device)}

        y_key = y_gmm[trigger_mask].to(self.device)
        print(f"Trigger set size: {x_key['input_ids'].shape[0]}")

        if x_key["input_ids"].shape[0] == 0:
            print("Warning: Empty trigger set, using all samples")
            x_key = trigger_inputs

            y_key = y_gmm.to(self.device)

        # 5. Generate watermark and matrix_a
        watermark = torch.randint(0, 2, (n_wat, self.config["watermark_len"])).float().to(self.device)
        matrix_a = torch.randn(n_features, self.config["watermark_len"]).to(self.device)
        print(f"Watermark shape: {watermark.shape}")

        
        ## 6. Initialize DeepSigns module with GMM means of carrier classes
        mu_carriers = gmm.mu[watermarked_classes.cpu()].to(self.device)
        deepsigns_module = DeepSignsModule(mu_carriers).to(self.device)

        # Non-carrier means (frozen)
        non_carrier_idx = [i for i in range(self.config["n_components"]) if i not in watermarked_classes.tolist()]
        mu_bar = gmm.mu[non_carrier_idx].to(self.device).detach()

        
        # 4. Optimiseurs
        optimizer = torch.optim.AdamW([
            {'params': watermarked_model.parameters(), 'lr': self.config["lr"]},
            {'params': deepsigns_module.parameters(), 'lr': self.config["lr_ds"]}
        ])
        
        ce_loss = nn.CrossEntropyLoss()
        bce_loss = nn.BCELoss()

        # Hook for activation extraction during training
        target_layer = self._get_target_layer(watermarked_model, self.config["layer_name"])


        # 5. Boucle d'entrainement
        for epoch in range(self.config["epochs"]):
            pbar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{self.config['epochs']}")
            correct = 0
            total = 0
            
            for batch in pbar:
                input_ids = batch["input_ids"].to(self.device)
                attention_mask = batch["attention_mask"].to(self.device)
                labels = batch["labels"].to(self.device)
                

                optimizer.zero_grad()
                
                # Tache principale (classification)
                outputs = watermarked_model(input_ids=input_ids, attention_mask=attention_mask)
                l_main = ce_loss(outputs.logits, labels)
                
                # Tache watermark (sur trigger set)
                hook_wat = FeatureHook(target_layer)
                _ = watermarked_model(
                    input_ids=x_key["input_ids"],
                    attention_mask=x_key["attention_mask"]
                )
                act = hook_wat.features
                if len(act.shape) == 4:
                    act = act.mean(dim=(2, 3))
                elif len(act.shape) == 3:
                    act = act.mean(dim=(1))
                hook_wat.close()

                
                # C. GMM Loss
                mu_dp = deepsigns_module.var_param
                gmm_loss, sep_loss = self._mu_loss(act, mu_dp, mu_bar, watermarked_classes, y_key)
                l_mu = gmm_loss - sep_loss

                # D. Watermark Loss
                matrix_g = deepsigns_module(matrix_a)
                l_wat = bce_loss(matrix_g, watermark)

                mu_stat = torch.stack([act[y_key == t].mean(dim=0) for t in watermarked_classes])
                l_align = F.mse_loss(mu_stat, deepsigns_module.var_param)

                # Total loss
                # l_total = l_main + self.config["lambda_1"] * l_mu + self.config["lambda_2"] * l_wat
                l_total = l_main + gmm_loss + l_align - self.config["lambda_1"] * sep_loss + self.config["lambda_2"] * l_wat

                l_total.backward()
                optimizer.step()

                

                
                # Metrics
                preds = torch.argmax(outputs.logits, dim=-1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
                
                ber = self._compute_ber(matrix_g, watermark)
                acc = 100. * correct / total
                pbar.set_postfix(L_Main=f"{l_main.item():.3f}", L_Mu= f"{l_mu.item():.3f}",  L_Wat=f"{l_wat.item():.3f}", BER=f"{ber:.2f}", Acc=f"{acc:.1f}%")
            
            # Evaluation en fin d'epoch
            if eval_dataloader:
                eval_acc = self.evaluate(watermarked_model, eval_dataloader)
                print(f"Epoch {epoch+1} | Eval Accuracy: {eval_acc:.2f}%")
        
        # Sauvegarde des cles
        self.saved_keys = {
            "watermark": watermark,
            "matrix_a": matrix_a,
            "watermarked_classes": watermarked_classes,
            "watermarked_model": watermarked_model,
            "deepsigns_module": deepsigns_module,
            "x_key": x_key,
            "y_key": y_key,
        }
        
        final_ber, _ = self.extract(watermarked_model)
        print(f"\n--- Embedding termine | BER final: {final_ber:.2f} ---")
        return watermarked_model
    
    def evaluate(self, model, dataloader):
        """Evalue la precision du modele."""
        model.eval()
        correct = 0
        total = 0
        
        with torch.no_grad():
            for batch in dataloader:
                input_ids = batch["input_ids"].to(self.device)
                attention_mask = batch["attention_mask"].to(self.device)
                labels = batch["labels"].to(self.device)
                
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                preds = torch.argmax(outputs.logits, dim=-1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        
        model.train()
        return 100. * correct / total
    
    def extract(self, suspect_model=None):
        """
        Extrait la marque d'un modele suspect.
        """
        if suspect_model is None:
            suspect_model = self.saved_keys["watermarked_model"]
        watermark = self.saved_keys["watermark"]
        matrix_a = self.saved_keys["matrix_a"]
        watermarked_classes = self.saved_keys["watermarked_classes"]
        x_key = self.saved_keys["x_key"]

        y_key = self.saved_keys["y_key"]




        
        # Hook sur le modele suspect
        target_layer = self._get_target_layer(suspect_model, self.config["layer_name"])
        hook = FeatureHook(target_layer)
        suspect_model.eval()
        
        # Passage du trigger set
        with torch.no_grad():
            _ = suspect_model(input_ids=x_key["input_ids"], attention_mask=x_key["attention_mask"])
            act = hook.features
            if len(act.shape) == 4:
                act = act.mean(dim=(2, 3))
            elif len(act.shape) == 3:
                act = act.mean(dim=(1))
        hook.close()

        mu_ext = torch.stack([
            act[y_key == t].mean(dim=0) if (y_key == t).sum() > 0 else torch.zeros(act.shape[1], device=self.device)
            for t in watermarked_classes
        ])

        # Project through matrix_a
        g_ext = torch.sigmoid(mu_ext @ matrix_a)
        ber = self._compute_ber(g_ext, watermark)

        print(f"BER Extrait : {ber:.2f}")
        return ber, g_ext
    
    @staticmethod
    def _compute_ber(pred, target):
        return ((pred > 0.5).float() != target).float().mean().item()

print("Classes chargees (DeepSignSentiment, FeatureHook, ProjectionNet)")

/home/latim/PycharmProjects/WatDNN/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Classes chargees (DeepSignSentiment, FeatureHook, ProjectionNet)


In [2]:
# --- Chargement des donnees SST-2 ---

# Charger le dataset SST-2 (sentiment binaire)
dataset = load_dataset("glue", "sst2")

# Initialiser le watermarker
deepsign = DeepSignSentiment(device="cuda" if torch.cuda.is_available() else "cpu")

# Afficher les couches disponibles
deepsign.list_layers()

# Tokenization
def tokenize_function(examples):
    return deepsign.tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# DataLoaders
train_dataloader = DataLoader(tokenized_dataset["train"], batch_size=64, shuffle=True)
eval_dataloader = DataLoader(tokenized_dataset["validation"], batch_size=64)

print(f"\nTrain samples: {len(tokenized_dataset['train'])}")
print(f"Eval samples: {len(tokenized_dataset['validation'])}")

Loading weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 2958.44it/s, Materializing param=bert.pooler.dense.weight]
BertForSequenceClassification LOAD REPORT from: prajjwal1/bert-tiny
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.decoder.weight             | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.decoder.bias               | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
bert.embeddings.position_ids      

Couches disponibles (modules):
  bert
  bert.embeddings
  bert.embeddings.word_embeddings
  bert.embeddings.position_embeddings
  bert.embeddings.token_type_embeddings
  bert.embeddings.LayerNorm
  bert.embeddings.dropout
  bert.encoder
  bert.encoder.layer
  bert.encoder.layer.0
  bert.encoder.layer.0.attention
  bert.encoder.layer.0.attention.self
  bert.encoder.layer.0.attention.self.query
  bert.encoder.layer.0.attention.self.key
  bert.encoder.layer.0.attention.self.value
  bert.encoder.layer.0.attention.self.dropout
  bert.encoder.layer.0.attention.output
  bert.encoder.layer.0.attention.output.dense
  bert.encoder.layer.0.attention.output.LayerNorm
  bert.encoder.layer.0.attention.output.dropout
  bert.encoder.layer.0.intermediate
  bert.encoder.layer.0.intermediate.dense
  bert.encoder.layer.0.intermediate.intermediate_act_fn
  bert.encoder.layer.0.output
  bert.encoder.layer.0.output.dense
  bert.encoder.layer.0.output.LayerNorm
  bert.encoder.layer.0.output.dropout
  bert.enc

Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1821/1821 [00:00<00:00, 33708.15 examples/s]


Train samples: 67349
Eval samples: 872


In [3]:


# Lancer l'embedding
watermarked_model = deepsign.embed(
    train_dataloader, 
    eval_dataloader,
    trigger_dataset=tokenized_dataset["train"]
)

--- Demarrage Embedding DeepSign (bert.encoder.layer.1.output) ---
Extracting activations for GMM...
torch.Size([64, 128, 128])
torch.Size([64, 128, 128])
torch.Size([64, 128, 128])
torch.Size([64, 128, 128])
Activation shape: torch.Size([256, 128])
Fitting GMM...
Watermarked classes: [0, 1]
Trigger set size: 122
Watermark shape: torch.Size([2, 64])


Epoch 1/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:09<00:00, 113.41it/s, Acc=80.7%, BER=0.01, L_Main=0.323, L_Mu=-0.383, L_Wat=0.851]


Epoch 1 | Eval Accuracy: 81.31%


Epoch 2/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:09<00:00, 112.38it/s, Acc=88.0%, BER=0.01, L_Main=0.382, L_Mu=-0.361, L_Wat=0.799]


Epoch 2 | Eval Accuracy: 82.80%


...

Epoch 28 | Eval Accuracy: 80.39%


Epoch 29/30: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:09<00:00, 107.41it/s, Acc=98.3%, BER=0.00, L_Main=0.023, L_Mu=-2.121, L_Wat=0.001]


Epoch 29 | Eval Accuracy: 79.82%


Epoch 30/30: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:09<00:00, 106.74it/s, Acc=98.4%, BER=0.00, L_Main=0.007, L_Mu=-2.190, L_Wat=0.001]


Epoch 30 | Eval Accuracy: 78.44%
BER Extrait : 0.00

--- Embedding termine | BER final: 0.00 ---


In [4]:
# --- Test d'Extraction ---

print("=" * 50)
print("TEST D'EXTRACTION")
print("=" * 50)

# Extraction sur le modele watermarke
print("\n1. Extraction sur le modele watermarke:")
ber_watermarked, _ = deepsign.extract(watermarked_model)

# Extraction sur un modele vierge (non-watermarke)
print("\n2. Extraction sur un modele vierge (devrait avoir BER ~0.5):")
model_id="prajjwal1/bert-tiny"
config = BertConfig.from_pretrained(model_id, num_labels=2)
fresh_model = BertForSequenceClassification.from_pretrained(model_id, config=config).to(device=deepsign.device)

ber_fresh, _ = deepsign.extract(fresh_model)

print(f"\nResume:")
print(f"  - BER modele watermarke: {ber_watermarked:.2f} (attendu: ~0.0)")
print(f"  - BER modele vierge: {ber_fresh:.2f} (attendu: ~0.5)")

TEST D'EXTRACTION

1. Extraction sur le modele watermarke:
BER Extrait : 0.00

2. Extraction sur un modele vierge (devrait avoir BER ~0.5):


Loading weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 1559.04it/s, Materializing param=bert.pooler.dense.weight]
BertForSequenceClassification LOAD REPORT from: prajjwal1/bert-tiny
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.decoder.weight             | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.decoder.bias               | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
bert.embeddings.position_ids      

BER Extrait : 0.48

Resume:
  - BER modele watermarke: 0.00 (attendu: ~0.0)
  - BER modele vierge: 0.48 (attendu: ~0.5)


In [7]:



def run_distillation_attack(deepsign_obj, train_dataloader, eval_dataloader, epochs=5, lr=5e-5):
    """
    Tente de transferer la fonctionnalite du modele watermarke vers un modele vierge.
    Verifie si la marque (basee sur les activations) survit.
    """
    device = deepsign_obj.device
    
    # 1. Teacher (Gele)
    teacher_model = deepsign_obj.saved_keys["watermarked_model"]
    teacher_model.eval()
    for p in teacher_model.parameters():
        p.requires_grad = False
    
    # 2. Student (Vierge - Meme architecture)
    config = BertConfig.from_pretrained(model_id, num_labels=2)
    print("\n--- Initialisation du Student ---")
    student_model = BertForSequenceClassification.from_pretrained(model_id, config=config).to(device)
    student_model.train()
    
    # Verification initiale
    print("\n[Check] BER Teacher:")
    teacher_ber, _ = deepsign_obj.extract(teacher_model)
    print("[Check] BER Student (Avant):")
    student_ber, _ = deepsign_obj.extract(student_model)
    
    optimizer = torch.optim.AdamW(student_model.parameters(), lr=lr)
    history = {"loss": [], "ber": [], "acc": []}
    
    print(f"\n--- Distillation ({epochs} epochs) ---")
    feature_criterion = nn.MSELoss()
    for epoch in range(epochs):
        pbar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")
        running_loss = 0.0
        
        student_model.train()
        
        for batch in pbar:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            
            optimizer.zero_grad()
            teacher_layer = deepsign._get_target_layer(teacher_model, deepsign.config["layer_name"])
            hook = FeatureHook(teacher_layer)
            # Output Matching (Soft Labels)
            with torch.no_grad():
                teacher_logits = teacher_model(input_ids=input_ids, attention_mask=attention_mask).logits
                teacher_probs = F.softmax(teacher_logits / 2.0, dim=-1)
                t_features = hook.features
            hook.close()

            student_layer = deepsign._get_target_layer(student_model, deepsign.config["layer_name"])
            hook = FeatureHook(student_layer)
            student_logits = student_model(input_ids=input_ids, attention_mask=attention_mask).logits
            student_log_probs = F.log_softmax(student_logits / 2.0, dim=-1)
            s_features = hook.features
            hook.close()
            # print(f"Teacher features shape: {t_features.shape} | Student features shape: {s_features.shape}")
            # t_mean, t_var = compute_stats(t_features)
            # s_mean, s_var = compute_stats(s_features)
            loss_feat = feature_criterion(s_features.mean(dim=[0, 1]), t_features.mean(dim=[0, 1])) + feature_criterion(s_features.var(dim=[0, 1]), t_features.var(dim=[0, 1]))


            # KL Divergence Loss
            loss_kl = F.kl_div(student_log_probs, teacher_probs, reduction="batchmean") * (2.0 ** 2)
            loss= loss_kl + loss_feat
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            pbar.set_postfix(Loss=f"{loss.item():.4f}")
        
        # Evaluation
        avg_loss = running_loss / len(train_dataloader)
        history["loss"].append(avg_loss)
        
        current_ber, pred_wm_prob= deepsign_obj.extract(student_model)
        history["ber"].append(current_ber)
        
        eval_acc = deepsign_obj.evaluate(student_model, eval_dataloader)
        history["acc"].append(eval_acc)
        
        print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f} | BER: {current_ber:.2f} | Acc: {eval_acc:.1f}% err_wat: {nn.BCELoss()(pred_wm_prob, deepsign_obj.saved_keys["watermark"]).item()}")
    
    return student_model, history

In [8]:
# --- Lancer l'attaque par distillation ---

student_model, attack_history = run_distillation_attack(
    deepsign,
    train_dataloader, 
    eval_dataloader,
    epochs=1000
)


--- Initialisation du Student ---


Loading weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 1706.42it/s, Materializing param=bert.pooler.dense.weight]
BertForSequenceClassification LOAD REPORT from: prajjwal1/bert-tiny
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.decoder.weight             | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.decoder.bias               | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
bert.embeddings.position_ids      


[Check] BER Teacher:
BER Extrait : 0.00
[Check] BER Student (Avant):
BER Extrait : 0.48

--- Distillation (1000 epochs) ---


Epoch 1: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:05<00:00, 184.50it/s, Loss=0.9723]


BER Extrait : 0.20
Epoch 1 | Loss: 1.5939 | BER: 0.20 | Acc: 81.1% err_wat: 2.260035753250122


...

BER Extrait : 0.00
Epoch 159 | Loss: 0.0111 | BER: 0.00 | Acc: 79.7% err_wat: 0.006378501653671265


Epoch 160: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:06<00:00, 164.85it/s, Loss=0.0056]


BER Extrait : 0.01
Epoch 160 | Loss: 0.0113 | BER: 0.01 | Acc: 79.9% err_wat: 0.009715180844068527


Epoch 161:  31%|████████████████████████████████████████████████████████████████████████████                                                                                                                                                                      | 331/1053 [00:01<00:04, 166.68it/s, Loss=0.0074]


KeyboardInterrupt: 

In [ ]:
# --- Visualisation des Resultats ---

import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Loss
axes[0].plot(attack_history["loss"], 'b-o')
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Distillation Loss")
axes[0].set_title("Loss durant Distillation")
axes[0].grid(True)

# BER
axes[1].plot(attack_history["ber"], 'r-o')
axes[1].axhline(y=0.5, color='gray', linestyle='--', label='Random (0.5)')
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("BER")
axes[1].set_title("BER du Student (Watermark)")
axes[1].set_ylim([0, 1])
axes[1].legend()
axes[1].grid(True)

# Accuracy
axes[2].plot(attack_history["acc"], 'g-o')
axes[2].set_xlabel("Epoch")
axes[2].set_ylabel("Accuracy (%)")
axes[2].set_title("Accuracy du Student")
axes[2].grid(True)

plt.tight_layout()
plt.show()

# Resume final
print("\n" + "=" * 50)
print("RESUME FINAL")
print("=" * 50)
teacher_ber, _ = deepsign.extract(deepsign.saved_keys['watermarked_model'])
print(f"BER Teacher (watermark): {teacher_ber:.2f}")
print(f"BER Student (apres distillation): {attack_history['ber'][-1]:.2f}")
print(f"Accuracy Student: {attack_history['acc'][-1]:.1f}%")

if attack_history['ber'][-1] > 0.4:
    print("\n-> La marque DeepSigns n'a PAS survecu a la distillation (BER eleve)")
else:
    print("\n-> La marque Deepsigns a survecu a la distillation (BER faible)")